# Modellbildung

In [ ]:
import sys
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp

Ziel ist die Modellierung des Eingangs-Ausgangsverhaltens des bekannten 2-Tanksystems.
Dazu sollen drei verschiedene Varianten verglichen werden:

1. Die Zeitkonstanten des System lassen sich aus den physikalischen Parametern und der Ruhelage bestimmen.
2. Messung des Eingang-Ausgangsverhaltens des Gesamtsystems
3. Aufspaltung des Systems in zwei Teilsysteme und Bestimmung der Zeitkonstanten durch getrennte Betrachtung.

Aufbauend auf den analytisch gegebenen Werten und der Modellbildung aus der Übung, folgt nach Linearisierung ein System zweiter Ordnung in der Form
\begin{align*}
    T_1T_2\ddot{h}_2(t) + (T_1 + T_2)\dot{h}_2(t) + h_2(t) & = K_\text{P} u(t)
\end{align*}


### 1. Physikalische Modellierung

Tragen Sie hierzu in die Datei params.py die

In [ ]:
param_dict = readRst.readRst(datapath='../../../Dokumentation/tables/ParameterPidV1.rst')

Kp_analytic = param_dict['K']
T1_analytic = max(param_dict['Ti'], param_dict['Ts'])
T2_analytic = min(param_dict['Ti'], param_dict['Ts'])

display(Markdown(
   rf"""
from the **pid_design script** the individual parameters from modelling can be read out. This results in the time constants and the amplification as follows:

$K_p = {Kp_analytic}$

$T_1 = {T1_analytic}$

$T_2 = {T2_analytic}$
"""))

# Values from physical modeling
p_analytic = [Kp_analytic, T1_analytic, T2_analytic]

### 2. PT2 element from smoothed step response of the lower tank

Step response of PT2 element - creep case (page 18 "EinführungRegelungstechnik" PDF):
\begin{align*}
h(t) = K_\text{P}\left(1 - \frac{1}{T_1-T_2}\left(T_1 e^{-\frac{t}{T_1}} - T_2 e^{-\frac{t}{T_2}}\right)\right)
\end{align*}

However, in this formula the input is set to 1. If the system is measured with a different input, this must be taken into account accordingly. In essence, we can simply divide the $K_p$ by $u_{in}$ and we get the corresponding $K_p$ for the set input voltage.

In [ ]:
# Laden der Messung
measDataPT2 = pd.read_csv('Sprungantwort_PT2_9V.csv')

In [ ]:
# Definition der Sprungantwort zur Bestimmung der Zeitkonstanten
def pt2Analytic(t, Kp, T1, T2):
    val = Kp * (1 - 1 / (T1 - T2) * (T1 * np.exp(-t/T1) - T2 * np.exp(-t/T2)))
    for i in range(len(val)):
        if not math.isfinite(val[i]):
            print(val[i])
            val[i] = 0
    return val

In [ ]:
x_data = measDataPT2['time']
y_data = measDataPT2['HeightT2']

# errorcorrect measured data (in HeightT2 some values are nan)
for i in range(len(y_data)):
   if not math.isfinite(y_data[i]):
      try:
         # if value is nan, use mean of prev and next value
         y_data[i] = (y_data[i-1] + y_data[i+1]) /2
      except: 
         print('implement more ')

# Delete part of the first second in which the pump voltage ramps up
time = 0.2
for i in range(len(x_data)):
   if x_data[i] >= time:
      # delete first entries, adapt indexes, set starttime = 0
      x_data = x_data.iloc[i:]
      x_data.index -= i
      x_data -= time
      # also delete corresponding height entries
      y_data = y_data.iloc[i:]
      y_data.index -= i
      break

# curvefitting... giving the analytic values as start values
p_opt, _ = curve_fit(pt2_analytic, xdata=x_data, ydata=y_data, p0=p_analytic)

# store fittet params:
Kp_pt2 = p_opt[0]
T1_pt2 = max(p_opt[1], p_opt[2]) # T1 > T2 is convention
T2_pt2 = min(p_opt[1], p_opt[2])

display(Markdown(
   rf"""
**Curve Fit** returns the values as follows. This results in the amplification and time constants as follows:

$K_p = {Kp_pt2}$

$T_1 = {T1_pt2}$

$T_2 = {T2_pt2}$
"""))

Vergleich der Ergebnisse mit Messung

In [ ]:
plt.close()
data.plot('time', 'HeightT2', label='measurement')
plt.plot(data['time'], pt2_analytic(data['time'], Kp_pt2, p_analytic[1], p_analytic[2]), label='modelled parameters')
# plt.plot(data['time'], pt2_analytic(data['time'], p_opt[0], p_opt[1], p_opt[2]), label='fitted parameters', color='r')

plt.legend()
plt.grid()
plt.show()

### 3. Serielle Verschaltung zweier PT$_1$ Elemente

Step response of PT1 element (page 16 "EinführungRegelungstechnik" PDF):

\begin{align}
h(t) = K_\text{P} \left(1 - e^{-\frac{t}{T_1}}\right)
\end{align}

In [ ]:
# defining the function according to:
# page 16 "EinführungRegelungstechnik" PDF
def pt1_analytic(t, Kp, T1):
    return Kp * (1 - np.exp(-t / T1))

#### Tank 1

In [ ]:
# Laden der Messung
data = pd.read_csv('../../../Messungen/2Tanksystem/20221122_Sprungantwort_9V_Aufbau1_V12_2_0vU_V21_2_1_vU.csv')

# curvefitting and printing the result
p_opt, _ = curve_fit(pt1_analytic, xdata=data['time'], ydata=data['HeightT1'])

# storing results
K_p1 = p_opt[0]
T_1 = p_opt[1]

display(Markdown(
   rf"""
**Curve Fit** returns the values as follows. This results in the time constant and the amplification as follows:

$K_p = {K_p1}$

$T_1 = {T_1}$
"""))

In [ ]:
# displaying the results
plt.close()

fig, ax = plt.subplots(tight_layout=True)
data.plot('time', 'HeightT1', ax=ax, label='measurement')
ax.plot(data['time'], pt1_analytic(data['time'], p_opt[0], p_opt[1]), label='fit')

ax.set_ylabel('Height [m]')
ax.set_xlabel('time [s]')
ax.legend()

plt.grid()
plt.show()

#### Tank 2

In [ ]:
# loading the data
data = pd.read_csv('../../../Messungen/2Tanksystem/20221025_Sprungantwort_Aufbau1_Pumpe_in_Tank2_V21_2_1vU_9V.csv')

# curvefitting and printing the result
p_opt, _ = curve_fit(pt1_analytic, xdata=data['time'], ydata=data['HeightT2'])

# storing results
K_p2 = p_opt[0]
T_2 = p_opt[1]

display(Markdown(
   rf"""
**Curve Fit** returns the values as follows. This results in the time constant and the amplification as follows:

$K_p = {K_p2}$

$T_1 = {T_2}$
"""))

In [ ]:
# displaying the results
plt.close()

fig, ax = plt.subplots(tight_layout=True)
data.plot('time', 'HeightT2', ax=ax, label='measurement')
ax.plot(data['time'], pt1_analytic(data['time'], p_opt[0], p_opt[1]), label='fit')

ax.set_ylabel('Height [m]')
ax.set_xlabel('time [s]')
ax.legend()

plt.grid()
plt.show()

#### PT2 parameter from the two PT1 elements

Kp von Gesamtsystem ist Kp2, also die Verstärkung vom zweiten Tank. Denn in der Ruhelage fließt in den ersten Tank genauso viel hinein wie hinaus. Da Kp im Grunde angibt wie sehr der Ausgang steigt, wenn man den Eingang erhöht, kann Kp2 als Kp für das gesamtsystem verwendet werden, da die Höhe des zweiten Tanks der Ausgang des Gesamtsystems ist.

In [ ]:
Kp_pt1 = K_p2
T1_pt1 = max(T_1, T_2)
T2_pt1 = min(T_1, T_2)

display(Markdown(
   rf"""
**Curve Fit** returns the values as follows. This results in the time constants and the amplification as follows:

$K_p = {Kp_pt1}$

$T_1 = {T1_pt1}$

$T_2 = {T2_pt1}$
"""))

## Vergleich aller Modelle

In [ ]:
params = {
    'T1': [round(T1_analytic, 5), round(T1_pt2, 5), round(T1_pt1, 5)],
    'T2': [round(T2_analytic, 5), round(T2_pt2, 5), round(T2_pt1, 5)],
    'Kp': [round(Kp_analytic, 5), round(Kp_pt2, 5), round(Kp_pt1, 5)],
}
print ("{:<5} {:<11} {:<9} {:<11}".format(' ','analytisch','PT2','PT1'))
for name, values in params.items():
    s1, s2, s3 = values
    print ("{:<5} {:<11} {:<9} {:<11}".format(name, s1, s2, s3))

In [ ]:
measDataPT2 = pd.read_csv('../../../Messungen/2Tanksystem/20221122_Sprungantwort_9V_Aufbau1_V12_2_0vU_V21_2_1vU.csv')
measDataPT1 = pd.read_csv('../../../Messungen/2Tanksystem/20221122_Sprungantwort_9V_Aufbau1_V12_2_0vU_V21_2_1vU.csv')

In [ ]:
def linSys(t, x, u, A, B):
    return A.dot(x) +  B.dot(u)

In [ ]:
A_analytic = np.array([[0, 1],
                       [-1 / (T1_analytic * T2_analytic), - (T1_analytic + T2_analytic) / (T1_analytic * T2_analytic)]])
B_analytic = np.array([[0],
                       [Kp_analytic / (T1_analytic * T2_analytic)]])

A_pt2 = np.array([[0, 1],
                  [-1 / (T1_pt2 * T2_pt2), - (T1_pt2 + T2_pt2) / (T1_pt2 * T2_pt2)]])
B_pt2 = np.array([[0],
                  [Kp_pt2 / (T1_pt2 * T2_pt2)]])

A_pt1 = np.array([[0, 1],
                  [-1 / (T1_pt1 * T2_pt1), - (T1_pt1 + T2_pt1) / (T1_pt1 * T2_pt1)]])
B_pt1 = np.array([[0],
                  [Kp_pt1 / (T1_pt1 * T2_pt1)]])

In [ ]:
timeDom = np.linspace(0, len(measData['HeightT2']), len(measData['HeightT2'])) / 10
x0 = [0, 0]
u = [1]

solAnalytic = solve_ivp(linSys, [tDom[0], tDom[-1]], x0, t_eval=tDom, args=(u, A_analytic, B_analytic))
solPT2 = solve_ivp(linSys, [tDom[0], tDom[-1]], x0, t_eval=tDom, args=(u, A_pt2, B_pt2))
solPT1 = solve_ivp(linSys, [tDom[0], tDom[-1]], x0, t_eval=tDom, args=(u, A_pt1, B_pt1))

In [ ]:
plt.close()

fig1, axes10 = plt.subplots(1, 1, sharex=True, figsize=(9,7))

axes10.plot(tDom, measDataPT2['HeightT2'], label=r'Messung PT$_2$')
axes10.plot(tDom, measDataPT1['HeightT2'], label=r'Messung PT$_1$')
axes10.plot(tDom, solAnalytisch.y[0], label='analytisch')
axes10.plot(tDom, solPT2.y[0], label=r'PT$_2$')
axes10.plot(tDom, solPT1.y[0], label=r'PT$_1$')

axes10.set_ylabel(r'T$_2$ Height [$m$]')
axes10.set_xlabel(r'time [$s$]')

handlesAx1, labelsAx1 = axes10.get_legend_handles_labels()
axes10.legend([handle for i, handle in enumerate(handlesAx1)],
              [label for i, label in enumerate(labelsAx1)],
              bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
              ncol=9, mode="expand", borderaxespad=0., framealpha=0.5)

axes10.grid()
plt.show()